In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision

In [2]:
#prepare transform

transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor()
])

In [3]:
# prepare datasets

BATCH_SIZE = 50

train_datasets = torchvision.datasets.CIFAR10(
    root="../data",
    train=True,
    transform=transform,
    download=True,
)

train_loader = torch.utils.data.DataLoader(
    train_datasets,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=2
)

In [4]:
#check the size of data

x_iter = iter(train_loader)
im, la = next(x_iter)

print(im.shape)
print(la.shape)
print(torch.unique(la))

print(f"Total training samples: {len(train_datasets)}")
print(f"Total batches: {len(train_loader)}")

torch.Size([50, 3, 32, 32])
torch.Size([50])
tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
Total training samples: 50000
Total batches: 1000


In [5]:
#make CNN model

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.fc1 = nn.Linear(64 * 8 * 8, 128)  # 修正: 64*8*8に変更
        self.fc2 = nn.Linear(128, 10)  # 10クラス分類
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # ✅ conv1 → ReLU → MaxPool
        x = self.pool(F.relu(self.conv2(x)))  # ✅ conv2 → ReLU → MaxPool

        x = x.view(x.size(0), -1)  # ✅ Flatten（batch_size, 64 * 8 * 8）
        
        x = F.relu(self.fc1(x))  # ✅ FC1 → ReLU
        x = self.fc2(x)  # ✅ FC2（出力層）

        return x

In [6]:
model = CNN()
criterion = nn.CrossEntropyLoss()  # 損失関数
optimizer = optim.Adam(model.parameters(), lr=0.001)  # 最適化手法


In [7]:
#define training function
from torch.utils.tensorboard import SummaryWriter

def train(model, trainloader, criterion, optimizer, num_epochs = 5, device = "cpu"):
    model.to(device)
    writer = SummaryWriter(log_dir="runs/cnn_exp_2025-02-25")
    
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0
        correct = 0
        total = 0
        
        for images, labels in trainloader:
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()
            output = model(images)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            _, predicted = torch.max(output, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
            
        epoch_loss = running_loss / len(trainloader)
        epoch_acc = correct / total # ✅ 修正
        print(f"Epoch {epoch+1}/{num_epochs} - Loss: {epoch_loss:.4f} - Accuracy: {epoch_acc:.2%}")
        
        #add info on tensorboard
        writer.add_scalar("Loss/train", epoch_loss, epoch)
        writer.add_scalar("Accuracy/train", epoch_acc, epoch)
        for name, param in model.named_parameters():
            writer.add_histogram(f"weights/{name}", param, epoch)
            writer.add_histogram(f"grads/{name}", param.grad, epoch)
    writer.close()
    print("training finish")

In [8]:
train(model, train_loader, criterion, optimizer, num_epochs=5 )

Epoch 1/5 - Loss: 1.4727 - Accuracy: 46.80%
Epoch 2/5 - Loss: 1.1197 - Accuracy: 60.19%
Epoch 3/5 - Loss: 0.9660 - Accuracy: 66.07%
Epoch 4/5 - Loss: 0.8695 - Accuracy: 69.50%
Epoch 5/5 - Loss: 0.7922 - Accuracy: 72.35%
training finish
